## Task: Résumé parsing and information extraction


### CoNVO

**Context:** Bloc is a career services management platform that builds smart career and data management tools for job-seekers and the organizations serving them. In particular, Bloc seeks to provide and facilitate access to tools for effectively presenting job-seekers' credentials and matching employers' job postings, and thereby improve outcomes.

**Need:** Many job-seekers come to Bloc's platform with a résumé already written. Forcing new users to re-enter all that information before they can utilize other tools (e.g. for résumé evaluation) is tedious, at best. This can also reduce time pressure during in-person sessions facilitated by Bloc, where every second counts.

**Vision:** Automated extraction of key information from existing résumés, submitted as PDFs while onboarding new users, in order to facilitate and streamline the process.

**Outcome:** A standalone, proof-of-concept process for extracting key résumé information and returning it as structured data, complete with unit tests and documentation on expected usage, limitations, and potential improvements.


### Data Summary

Bloc has provided ~125 résumés with a variety of styles, layouts, and contents, in PDF format. (Bonus: ~2400 résumés scraped from external sources...) Data quality seems good, and appears to be composed entirely of electronically-generated PDFs rather than (much more troublesome) scans of physical documents.

They typically include personal contact information, professional experience, education, and skills; they sometimes include information on other relevant experience (volunteering, leadership, side projects), professional and academic associations, honors and awards, and personal interests; they rarely include a professional objective / statement of purpose and references.

Since the amount of data is relatively small, and since résumés are so structured and standardized in terms of the information they include, a rules-based approach seems likely to succeed.


### Proposed Methodology

Cleanly extracting text from PDFs is tricky, since the format alters or throws out information for the sake of human-friendly layout, formatting, and such. Given this, it's best to use well-established tools for the text extraction, and highly accommodating parsing logic for the texts themselves. Rather than going full-bore on a complex, computer-vision based résumé parsing system, it'll be best to start with more foundational tools of text processing: regular expressions, fuzzy string matching, gazetteers/dictionaries, data sanitization/cleanup, and lots of trial-and-error.

See the code below for something to get you started.


### Definitions of Success

- **Baseline:** A function that accepts a résumé (TBD: as filepath or already-extracted text) and returns structured data for the most common résumé components: contact information, professional experience, education, and skills. The quality of the extracted values may be messy or not fully parsed, but shouldn't contain values for other components. Atypical résumé components may be skipped. This function should have basic unit tests and documentation.
- **Target:** A function that accepts a résumé and returns structured data for the most common résumé components (see Baseline), as well as other relevant experience, professional/academic associations, honors and awards, and personal interests. The quality of the extracted values should be almost fully parsed (e.g. no large blocks of relevant but unstructured text) and should not contain values for other components. Atypical résumé components may be skipped. This function should have unit tests covering a variety of expected scenarios and good documentation.
- **Stretch:** A function that accepts a résumé and returns structured data for any component that could be reasonably expected in such a document. The quality of the extracted values should be almost fully parsed (see Target). Particularly unusual résumé components may be skipped. This function should have comprehensive unit tests and documentation.

Note: We should try to get Bloc's buy-in / feedback on a schema, since they already ingest and store some of this data in their systems.


### Risks

It's possible that the information included in / extracted from résumés is too complex or varied for sufficiently accurate rules-based parsing, in which case a more sophisticated (ML- or DL-based) approach would be necessary, albeit impractical owing to time and data constraints. It's also possible that a rules-based approach is feasible, but too difficult / large a task for a single day's work.

Another separate risk deals with personally-identifiable information (PII), which is intrinsic to a résumé, but which DataKind typically prefers to strip out of the data assigned to volunteers. A relatively practical solution would entail extracting text from the PDFs beforehand, then replacing direct PII (name and contact info) with placeholder values, but we'd still have volunteers working with indirect PII such as education / employment history. DataKind may not be able to abide such a middle ground.

## Source Code

In [1]:
%load_ext watermark

### Utils

In [2]:
import re

In [3]:
RE_NAME = re.compile(r"^(([(\"][A-Z]\w+[)\"]|[A-Z]\w+|[A-Z])[.,]?[ -]?){2,5}$", flags=re.UNICODE)
RE_URL = re.compile(
    r"(?:^|(?<![\w/.]))"
    # protocol identifier
    # r"(?:(?:https?|ftp)://)"  <-- alt?
    r"(?:(?:https?://|ftp://|www\d{0,3}\.))"
    # user:pass authentication
    r"(?:\S+(?::\S*)?@)?"
    r"(?:"
    # IP address exclusion
    # private & local networks
    r"(?!(?:10|127)(?:\.\d{1,3}){3})"
    r"(?!(?:169\.254|192\.168)(?:\.\d{1,3}){2})"
    r"(?!172\.(?:1[6-9]|2\d|3[0-1])(?:\.\d{1,3}){2})"
    # IP address dotted notation octets
    # excludes loopback network 0.0.0.0
    # excludes reserved space >= 224.0.0.0
    # excludes network & broadcast addresses
    # (first & last IP address of each class)
    r"(?:[1-9]\d?|1\d\d|2[01]\d|22[0-3])"
    r"(?:\.(?:1?\d{1,2}|2[0-4]\d|25[0-5])){2}"
    r"(?:\.(?:[1-9]\d?|1\d\d|2[0-4]\d|25[0-4]))"
    r"|"
    # host name
    r"(?:(?:[a-z\u00a1-\uffff0-9]-?)*[a-z\u00a1-\uffff0-9]+)"
    # domain name
    r"(?:\.(?:[a-z\u00a1-\uffff0-9]-?)*[a-z\u00a1-\uffff0-9]+)*"
    # TLD identifier
    r"(?:\.(?:[a-z\u00a1-\uffff]{2,}))"
    r")"
    # port number
    r"(?::\d{2,5})?"
    # resource path
    r"(?:/\S*)?"
    r"(?:$|(?![\w?!+&/]))",
    flags=re.UNICODE | re.IGNORECASE,
)
RE_SHORT_URL = re.compile(
    r"(?:^|(?<![\w/.]))"
    # optional scheme
    r"(?:(?:https?://)?)"
    # domain
    r"(?:\w-?)*?\w+(?:\.[a-z]{2,12}){1,3}"
    r"/"
    # hash
    r"[^\s.,?!'\"|+]{2,12}"
    r"(?:$|(?![\w?!+&/]))",
    flags=re.UNICODE | re.IGNORECASE,
)
RE_EMAIL = re.compile(
    r"(?:mailto:)?"
    r"(?:^|(?<=[^\w@.)]))([\w+-](\.(?!\.))?)*?[\w+-]@(?:\w-?)*?\w+(\.([a-z]{2,})){1,3}"
    r"(?:$|(?=\b))",
    flags=re.UNICODE | re.IGNORECASE,
)
RE_PHONE_NUMBER = re.compile(
    # core components of a phone number
    r"(?:^|(?<=[^\w)]))(\+?1[ .-]?)?(\(?\d{3}\)?[ .-]?)?(\d{3}[ .-]?\d{4})"
    # extensions, etc.
    r"(\s?(?:ext\.?|[#x-])\s?\d{2,6})?(?:$|(?=\W))",
    flags=re.UNICODE | re.IGNORECASE,
)
RE_STREET_ADDRESS = re.compile(
    # r"[ \w]{3,}([A-Za-z]\.)?([ \w]*\#\d+)?,?[ \w]{3,}, [A-Za-z]{2} \d{5}(-\d{4})?",
    r"(\d+ ((?! \d+ ).)*?) [A-Za-z]{2} \d{5}(-\d{4})?",
    flags=re.UNICODE,
)

In [4]:
def get_filepaths(dirpath, suffixes):
    """
    Get full paths to all files under ``dirpath``
    with a file type in ``suffixes``.

    Args:
        dirpath (:class:`pathlib.Path`)
        suffixes (Set[str])
    
    Returns:
        List[str]
    """
    return sorted(
        str(path) for path in dirpath.resolve().iterdir()
        if path.is_file() and
        path.suffix in suffixes
    )

### Dataset Generation

**Note:** Don't run this section! (Besides, you _can't_, because the raw data has not been made available to you.) This is just to help you understand the data's provenance. Instead, download the already-generated datasets from OneDrive. (There's a link on the Bloc Project Home document in Dropbox.)

In [5]:
import io
import operator
import os
import pathlib
import shutil
import zipfile

import ftfy
from faker import Faker
from toolz import itertoolz

In [6]:
%watermark -v -iv

ftfy  5.6
re    2.2.1
toolz 0.10.0
CPython 3.7.4
IPython 7.8.0


In [7]:
FAKER = Faker(local="en_US")

In [8]:
out_dirpath = pathlib.Path("/Users/burtondewilde/Desktop/datakind/bloc/msvdd_Bloc/data/resumes")

In [9]:
def replace_pii(text):
    """
    Replace personally-identifying information in ``text``
    with randomly generated fake equivalents.
    
    Args:
        text (str)
    
    Returns:
        str
    """
    # let's start with names, which are usually on the first line    
    first_line, *the_rest = text.split("\n", maxsplit=1)
    first_line = RE_NAME.sub(FAKER.name(), first_line.strip())
    text = "\n".join([first_line] + the_rest)
    # next, let's replace emails, urls, and addresses
    # which are usually in the first "chunk" of info
    # first_chunk, *the_rest = re.split(r"\n{2,}", text, maxsplit=1)
    first_chunk = text[:150]
    the_rest = text[150:]
    first_chunk = RE_PHONE_NUMBER.sub(FAKER.phone_number(), first_chunk)
    first_chunk = RE_EMAIL.sub(FAKER.email(), first_chunk)
    first_chunk = RE_URL.sub(FAKER.url(), first_chunk)
    first_chunk = RE_STREET_ADDRESS.sub(FAKER.address().replace("\n", " "), first_chunk)
    text = first_chunk + the_rest
    return text

#### Bloc Fellows' résumés

In [10]:
in_fellows_dirpath = pathlib.Path("/Users/burtondewilde/Desktop/datakind/bloc/raw_data/resumes/fellows")
filepaths = get_filepaths(in_fellows_dirpath, {".pdf"})
print("# files:", len(filepaths))
filepaths[:3]

# files: 128


['/Users/burtondewilde/Desktop/datakind/bloc/raw_data/resumes/fellows/2018FellowsResumes[002-002].pdf',
 '/Users/burtondewilde/Desktop/datakind/bloc/raw_data/resumes/fellows/2018FellowsResumes[003-003].pdf',
 '/Users/burtondewilde/Desktop/datakind/bloc/raw_data/resumes/fellows/2018FellowsResumes[004-004].pdf']

#### (a caveat)

Programmatically extracting text from a PDF with an atypical layout — such as a résumé — is _tricky_. Mistakes happen, and the results aren't always consistent with how a human would type it out.

I tried several options (see below). The Python binding to Apache Tika (`python-tika`) seemed to give the nicest text extractions, although the JVM dependency is unfortunate. `textract` provides a convenient and consistent interface, but results are mediocre and installation involves a lot of extra packages. `pdfminer` and its many forks are highly customizable, but confusing to use and, to be honest, a hot mess as far as code quality goes. I'm surprised Python doesn't have a better solution to this problem, but _whatchagonnado_.

In [11]:
def extract_text_from_pdf(filepath, *, min_len):
    """
    Extract text from a PDF at ``filepath`` using the first package
    to get the job done in extracting at least ``min_len`` chars.
    
    Args:
        filepath (str)
        min_len (int)
    
    Returns:
        str
    """
    text = ""
    funcs = [extract_text_tika, extract_text_pdfminer, extract_text_textract]
    for func in funcs:
        _text = func(filepath)
        if len(_text) >= min_len:
            text = _text
            break
    return text


def extract_text_textract(filepath):
    """
    Extract text from a PDF at ``filepath`` using ``textract`` + ``pdftotext``.
    
    Args:
        filepath (str)
    
    Returns:
        str
    """
    # hiding the import so folks don't have to worry about installing it
    import textract
    
    return textract.process(
        filepath, method="pdftotext", encoding="utf-8"
    ).decode("utf-8").strip()


def extract_text_pdfminer(filepath):
    """
    Extract text from a PDF at ``filepath`` using ``yapdfminer``.

    Args:
        filepath (str)
    
    Returns:
        str
    """
    # hiding the import so folks don't have to worry about installing it
    import pdfminer.converter
    import pdfminer.layout
    import pdfminer.pdfinterp
    import pdfminer.pdfpage
    
    laparams = pdfminer.layout.LAParams(boxes_flow=0.5)
    retstr = io.StringIO()
    rsrcmgr = pdfminer.pdfinterp.PDFResourceManager()
    device = pdfminer.converter.TextConverter(
        rsrcmgr, retstr, codec="utf-8", laparams=laparams)
    interpreter = pdfminer.pdfinterp.PDFPageInterpreter(rsrcmgr, device)
    fp = io.open(filepath, mode="rb")
    for page in pdfminer.pdfpage.PDFPage.get_pages(fp, set(), maxpages=0, caching=True, check_extractable=True):
        interpreter.process_page(page)
    text = retstr.getvalue()
    fp.close()
    device.close()
    retstr.close()
    return text.strip()


def extract_text_tika(filepath):
    """
    Extract text from a PDF at ``filepath`` using ``python-tika``.
    
    Args:
        filepath (str)
    
    Returns:
        str
    """
    # hiding the import so folks don't have to worry about installing it
    from tika import parser
    
    result = parser.from_file(filepath)
    return result["content"].strip()


def clean_fellows_text(text):
    """
    Correct any encoding / mojibake / unicode weirdness, and remove
    superfluous numbering at the end of the résumé.
    
    Args:
        text (str)
    
    Returns:
        str
    """
    text = ftfy.fix_text(text)
    text = re.sub(r"\s+\d+\s*?$", "", text)
    return text

In [12]:
out_fellows_fpath = out_dirpath.joinpath("fellows_resumes.zip")
with zipfile.ZipFile(out_fellows_fpath, mode="w", compression=zipfile.ZIP_DEFLATED) as zf:
    for i, filepath in enumerate(filepaths):
        text = extract_text_from_pdf(filepath, min_len=150)
        if not text:
            print("unable to extract text from", filepath)
            continue
        text = clean_fellows_text(text)
        text = replace_pii(text)
        fname = "fellows_resume_{}.txt".format(i)
        zf.writestr(fname, text)

unable to extract text from /Users/burtondewilde/Desktop/datakind/bloc/raw_data/resumes/fellows/2018FellowsResumes[024-024].pdf
unable to extract text from /Users/burtondewilde/Desktop/datakind/bloc/raw_data/resumes/fellows/2018FellowsResumes[027-027].pdf


#### Amina's bonus scraped résumés

In [13]:
in_bonus_dirpath = pathlib.Path("/Users/burtondewilde/Desktop/datakind/bloc/raw_data/resumes/bonus")
filepaths = get_filepaths(in_bonus_dirpath, {".txt"})
print("# files:", len(filepaths))
filepaths[:3]

# files: 2432


['/Users/burtondewilde/Desktop/datakind/bloc/raw_data/resumes/bonus/_resume_ab0iv8_hopewell-fluorescence-stainless-steel-haledon-nj.txt',
 '/Users/burtondewilde/Desktop/datakind/bloc/raw_data/resumes/bonus/_resume_ab2abk_available-upon-request-givers-cmo-icd-new-york-ny.txt',
 '/Users/burtondewilde/Desktop/datakind/bloc/raw_data/resumes/bonus/_resume_ab3i5x_july-2013-5th-pharm-upon-request-harrison-nj-07029.txt']

#### (a caveat)

ADD CAVEAT HERE. (There is one.)

In [14]:
def clean_bonus_text(text):
    """
    Correct any encoding / mojibake / unicode weirdness, and remove
    superfluous "resumes (in)" lines at the end of the résumé.
    
    Args:
        text (str)
    
    Returns:
        str
    """
    text = ftfy.fix_text(text)
    text = re.sub(r"([\w]+ ?){1,3}resumes( in ([\w,]+ ?){1,3})?\n", "", text)
    return text

In [15]:
out_bonus_fpath = out_dirpath.joinpath("bonus_resumes.zip")
with zipfile.ZipFile(out_bonus_fpath, mode="w", compression=zipfile.ZIP_DEFLATED) as zf:
    for i, filepath in enumerate(filepaths):
        with io.open(filepath, mode="r") as f:
            text = f.read()
        if not text:
            print("unable to extract text from", filepath)
            continue
        text = clean_bonus_text(text)
        text = replace_pii(text)
        fname = "bonus_resume_{}.txt".format(i)
        zf.writestr(fname, text)

### Getting Started

In [12]:
import collections
import io
import operator
import pathlib

from toolz import itertoolz

In [13]:
%watermark -v -iv

ftfy  5.6
re    2.2.1
toolz 0.10.0
CPython 3.7.4
IPython 7.8.0


In [14]:
RE_BULLETS = re.compile(r"[\u25cf\u2022\u2023\u2043]", flags=re.UNICODE)
RE_BREAKING_SPACE = re.compile(r"(\r\n|[\n\v]){2,}", flags=re.UNICODE)
RE_NONBREAKING_SPACE = re.compile(r"[^\S\n\v]+", flags=re.UNICODE)
RE_MONTH = re.compile(
    r"(jan|january|feb|february|mar|march|apr|april|may|jun|june|jul|july|aug|august|sep|september|oct|october|nov|november|dec|december)",
    flags=re.IGNORECASE
)
RE_YEAR = re.compile(r"((19|20)\d{2})")

SECTION_HEADERS = {
    "education": {
        "education",
    },
    "experience": {
        "experience",
        "work experience",
        "professional experience",
        "work & research experience",
        "relevant experience",
        "experiences",
        "additional experience",
        "leadership",
        "leadership experience",
        "leadership and service",
    },
    "skills": {
        "skills",
        "technical skills",
        "skills & expertise",
        "technological skills",
        "tools",
        "languages",
        "programming languages",
        "languages and technologies",
        "language and technologies",
    },
    "achievements": {
        "achievements",
        "awards",
        "honors",
        "honors & awards",
        "honors, awards, and memberships",
        "fellowships & awards",
        "awards and certifications",
    },
    "projects": {
        "projects",
        "side projects",
        "technical projects",
        "programming projects",
        "github projects",
        "other projects",
    },
    "activities": {
        "activities",
        "volunteering",
        "activities and student groups",
    }
}

In [15]:
def load_resume_text(filepath):
    """
    Args:
        filepath (str)
    
    Returns:
        str
    """
    with io.open(filepath, mode="rt", encoding="utf-8") as f:
        text = f.read()
    return text


def preprocess_resume_text(text):
    """
    Args:
        text (str)
        
    Returns:
        str
    """
    # clean up weird stuff
    text = RE_BULLETS.sub("-", text)
    # normalize whitespace
    text = RE_NONBREAKING_SPACE.sub(" ", text).strip()
    text = RE_BREAKING_SPACE.sub(r"\n\n", text)
    # TODO: any other roughness that can be consistently smoothed out
    return text


def get_section_idxs(lines):
    """
    Args:
        lines (List[str])
    
    Returns:
        List[Tuple[str, int]]
    """
    section_idxs = [("START", 0)]
    for idx, line in enumerate(lines):
        for section, headers in SECTION_HEADERS.items():
            if (
                any(line.lower() == header for header in headers) or
                any(line.lower().startswith(header + ":") for header in headers)
            ):
                section_idxs.append((section, idx))
    section_idxs.append(("END", len(lines)))
    return section_idxs


def get_section_lines(lines, section_idxs):
    """
    Args:
        lines (List[str])
        section_idxs (List[Tuple[str, int]])
    
    Returns:
        Dict[str, List[str]]
    """
    section_lines = collections.defaultdict(list)
    for (section, idx1), (_, idx2) in itertoolz.sliding_window(2, section_idxs):
        section_lines[section].extend(lines[idx1 : idx2])
    return dict(section_lines)


def parse_skills_section(lines):
    """
    Super rough example for extracting structured data from skills...
    
    Args:
        lines (List[str])
    
    Returns:
        Dict[str, List[str]]
    """
    skills = [
        skill.lstrip("- ")
        for line in lines
        for skill in re.split(r", +", line)
        if skill.strip() and
        skill.strip().lower() not in SECTION_HEADERS["skills"]
    ]
    return {"skills": skills}


def parse_education_section(lines):
    raise NotImplementedError("Greetings, DataDiver!")
    

# and so on and so forth

In [16]:
data_dir = pathlib.Path("/Users/burtondewilde/Desktop/datakind/bloc/msvdd_Bloc/data/resumes")
filepaths = get_filepaths(data_dir, {".txt"})
print("# files:", len(filepaths))
filepaths[:3]

# files: 126


['/Users/burtondewilde/Desktop/datakind/bloc/msvdd_Bloc/data/resumes/resume_0.txt',
 '/Users/burtondewilde/Desktop/datakind/bloc/msvdd_Bloc/data/resumes/resume_1.txt',
 '/Users/burtondewilde/Desktop/datakind/bloc/msvdd_Bloc/data/resumes/resume_10.txt']

In [17]:
for filepath in filepaths:
    text = load_resume_text(filepath)
    text = preprocess_resume_text(text)
    lines = [line.strip() for line in text.split("\n")]
    section_idxs = get_section_idxs(lines)
    section_lines = get_section_lines(lines, section_idxs)
    break

In [18]:
section_lines.get("skills", [])

['SKILLS',
 '',
 '- Java, Python, C/C++, C#, Git, MySQL',
 '- HTML, CSS, JavaScript, PHP, Sass, LESS',
 '',
 '- React, Angular, NodeJS',
 '- AWS, Drupal, WordPress']

In [19]:
parse_skills_section(section_lines.get("skills", []))

{'skills': ['Java',
  'Python',
  'C/C++',
  'C#',
  'Git',
  'MySQL',
  'HTML',
  'CSS',
  'JavaScript',
  'PHP',
  'Sass',
  'LESS',
  'React',
  'Angular',
  'NodeJS',
  'AWS',
  'Drupal',
  'WordPress']}

_just curious_: what are the most common section headers?

In [20]:
header_counts = collections.Counter()
for filepath in filepaths:
    text = preprocess_resume_text(load_resume_text(filepath))
    lines = [line.strip() for line in text.split("\n")]
    section_idxs = get_section_idxs(lines)
    header_counts.update(lines[idx].lower() for _, idx in section_idxs if idx != len(lines))
[item for item in header_counts.most_common() if item[1] > 1]

[('education', 86),
 ('projects', 45),
 ('skills', 44),
 ('experience', 41),
 ('work experience', 26),
 ('technical skills', 21),
 ('awards', 17),
 ('languages', 10),
 ('leadership', 9),
 ('professional experience', 8),
 ('achievements', 4),
 ('activities', 4),
 ('education:', 4),
 ('relevant experience', 4),
 ('leadership experience', 3),
 ('experience:', 3),
 ('volunteering', 2),
 ('experiences', 2),
 ('honors & awards', 2),
 ('additional experience', 2),
 ('languages and technologies', 2),
 ('programming projects', 2),
 ('projects:', 2),
 ('tools', 2),
 ('1', 2),
 ('side projects', 2)]